<a href="https://colab.research.google.com/github/Dilyar-Iskan/PythonCProfiler/blob/master/MTLab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pickle
import tensorflow as tf
import numpy as np
import time
import io
import os
import math

# Machine Translation Lab: todo
We ended up switching to Google Colab to utilize them GPU's.

**Important**
* replace greedy search with beam search
* rewrite BPE
* speeding up train_step

**If we have enough time**
* add dropout layers
* replace GRU with LSTM (apparently LSTM perform slightly better)
* add early stopping (as in checkpoint evaluation)
* we seem to delete the dot sometimes somehow



# Configuration
We used to have a config file but that's really inconvenient with google colab. 
If needed we would just replace everything below with configparser.

Data

In [ ]:
path_main_dir = "/content/drive/My Drive/Colab Notebooks/MT_Lab"

path_src_train = path_main_dir + "/multi30k.de"
path_tar_train = path_main_dir + "/multi30k.en"
path_src_val = path_main_dir + "/multi30k.dev.de"
path_tar_val = path_main_dir + "/multi30k.dev.en"

load_bpe_operations = False
bpe_operations = 7500

limit_training_data = 10000
batch_size = 256
buffer_size = limit_training_data

Model

In [ ]:
embedding_dimension = 256
encoder_units = 128
decoder_units = 128

Training

In [ ]:
learning_rate = 0.01
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

starting_epoch = 5
final_epoch = 5

Translation

In [ ]:
beam_width = 2

#Dictionary


In [ ]:
from collections import defaultdict


class Dictionary:
    def __init__(self, text):
        self.word_to_key = True
        self.vocab = defaultdict(int)
        i = 10  # 0-10 are special tokens
        for line in [line.split() for line in text]:
            for word in line:
                if word not in self.vocab:
                    self.vocab[word] = i
                    i += 1
        self.vocab["<UNK>"] = 0
        self.vocab["\n"] = 1

    def get_word(self, key):
        self.set_direction_key2word()
        return self.vocab[key]

    def get_key(self, word):
        self.set_direction_word2key()
        return self.vocab[word]

    def set_direction_word2key(self):
        if not self.word_to_key:
            self.word_to_key = True
            tmp = defaultdict(int)
            for key, word in self.vocab.items():
                tmp[word] = key
            self.vocab = tmp

    def set_direction_key2word(self):
        if self.word_to_key:
            self.word_to_key = False
            tmp = defaultdict(str)
            for word, key in self.vocab.items():
                tmp[key] = word
            self.vocab = tmp

    def to_array(self, text): # returns 2D list
        self.set_direction_word2key()
        ret = []
        for line in text:
            tmp = []
            for word in line.split():
                tmp.append(self.vocab[word])
            ret.append(tmp)
        return ret

    def to_text(self, array):
        self.set_direction_key2word()
        text = []
        for row in array:
            line = ""
            for key in row:
                line += self.vocab[key] + " "
            text.append(line[:-1] + "\n")
        return text


#Loading Data & Preprocessing


In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

### Byte Pair Encoding

In [ ]:
def bpe_train(text, number_of_merges):
    cnt = Counter()
    merges = []
    vocab = {}
    for line in text:
        for word in line.split():
            cnt[word] += 1
    for word, freq in cnt.items():
        vocab[" " + "  ".join(word) + "</w> "] = freq
    pair_freq = pair_freq_init(vocab)

    for _ in tqdm(range(number_of_merges)):
        (a, b) = pair_freq.most_common(1)[0][0]
        vocab = bpe_merge_vocab(vocab, pair_freq, a, b)
        merges.append((a, b))
    return merges

In [ ]:
def bpe_pair_freq_init(vocab):
    pairs = Counter()
    for word, freq in vocab.items():
        for a, b in zip(word.split()[:-1], word.split()[1:]):
            pairs[a, b] += freq
    return pairs

In [ ]:
def bpe_merge_vocab(vocab, pair_freq, a, b):
    prev = " " + a + "  " + b + " "
    cur = " " + a + b + " "
    new_vocab = {}
    for word, freq in vocab.items():
        merged = word.replace(prev, cur)
        if " " in merged[1:-1]:
            new_vocab[merged] = freq
            if merged != word:
                for a, b in zip(word.split()[:-1], word.split()[1:]):
                    pair_freq[a, b] -= freq
                for a, b in zip(merged.split()[:-1], merged.split()[1:]):
                    pair_freq[a, b] += freq
    del pair_freq[a, b]
    return new_vocab

bpe encode

In [ ]:
def bpe_encode(sentence, merge_opration):
   vocab = Dictionary(text)
    array = vocab.to_array(text)
    vocab.set_direction_key2word()
    for key in tqdm(vocab.vocab.keys()):
        word = "  ".join(vocab.get_word(key))
        word = " " + word + "</w> "
        for a, b in operations:
            if " " in word[1:-1]:
                word = word.replace(" " + a + "  " + b + " ", " " + a + b + " ")
            else:
                break
        word = "@@ ".join(word.split())
        word = word.replace("</w>", "")
        vocab.vocab[key] = word
    return vocab.to_text(array)

bpe decode

In [ ]:
def bpe_decode(sentence):
   ret = []
    for line in text:
        line = line.replace("@@ ", "")
        line = line.replace("@@.", ".")
        ret.append(line[:-1] + "\n")
    return ret


get merge operations

In [ ]:
def get_merge_operations(text, load_bpe_operations):
  if load_bpe_operations:
    pass
    # load
  else:
    pass
    # train

###Load Data

Load the Data from the Drive.
As of rn we do not use BPE because of irregularities in our algorithm.

In [ ]:
def load_data(path, sort = False):
  input = io.open(path, encoding='UTF-8').read().split('\n')
  # todo bpe encode the sentence here
  data = ["<s> " + sentence + " </s>" for sentence in input]
  return data

In [ ]:
src_train_txt = load_data(path_src_train)
tar_train_txt = load_data(path_tar_train)
src_val_txt = load_data(path_src_val)
tar_val_txt = load_data(path_tar_val)


In [ ]:
def sort_corpus(src_text, tar_text):
  c = zip(src_text, tar_text)
  c = sorted(c, key = lambda x: len(x[0])) 
  return zip(*c)


In [ ]:
src_train_txt, tar_train_txt = sort_corpus(src_train_txt, tar_train_txt)
src_train_txt = src_train_txt[:10000]
tar_train_txt = tar_train_txt[:10000]

Last element of each set

In [ ]:
pos = 6
print(src_train_txt[pos])
print(src_val_txt[pos])
print(tar_train_txt[pos])
print(tar_val_txt[pos])

<s> ein windhund rennt . </s>
<s> ein brauner hund rennt dem schwarzen hund hinterher . </s>
<s> a grayhound dog is sprinting . </s>
<s> a brown dog is running after the black dog . </s>


###Tokenize Data

In [ ]:
def create_token_dict(text):
    dict = tf.keras.preprocessing.text.Tokenizer(filters='')
    dict.fit_on_texts(text)
    return dict

In [ ]:
def get_tokenized_tensor(source_text, target_text, src_dict, tar_dict):
    tokenized_source = src_dict.texts_to_sequences(source_text)
    # post stands for padding after the sequence
    tokenized_source = tf.keras.preprocessing.sequence.pad_sequences(tokenized_source, padding='post')
    tokenized_target = tar_dict.texts_to_sequences(target_text)
    tokenized_target = tf.keras.preprocessing.sequence.pad_sequences(tokenized_target, padding='post')
    dataset = tf.data.Dataset.from_tensor_slices((tokenized_source, tokenized_target))
    return tokenized_source, tokenized_target

getting the dictionaries

In [ ]:
src_dict = create_token_dict(src_train_txt)
tar_dict = create_token_dict(tar_train_txt)

In [ ]:
src_train, tar_train = get_tokenized_tensor(src_train_txt, tar_train_txt, src_dict, tar_dict)

max_src_len = src_train.shape[1]
max_tar_len = tar_train.shape[1]

length of training data and length of longest sentence

In [ ]:
print(len(src_train))
print(len(tar_train))

print(max_src_len)
print(max_tar_len)


10000
10000
16
23


length of vocabulary:

In [ ]:
vocab_length_src = len(src_dict.index_word) + 1
vocab_length_tar = len(tar_dict.index_word) + 1

In [ ]:
print(vocab_length_src)
print(vocab_length_tar)

6909
4859


printing first element of sets

In [ ]:
print(src_train[pos])
print(src_dict.sequences_to_texts([src_train[pos]]))
print(tar_train[pos])
print(tar_dict.sequences_to_texts([tar_train[pos]]))

[   1    4 1088   71    3    2    0    0    0    0    0    0    0    0
    0    0]
['<s> ein windhund rennt . </s>']
[   2    1 2637   18    7 2638    4    3    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
['<s> a grayhound dog is sprinting . </s>']


### Create Dataset

In [ ]:
def get_dataset():
  return tf.data.Dataset.from_tensor_slices((src_train, tar_train)).shuffle(buffer_size).batch(batch_size, drop_remainder=True)

# Model


### Encoder
we're using GRU's rn but we'll have to switch them out for LSTM

***todo:*** reverse input (go backwards -> look at tf documentation)

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self):
      super(Encoder, self).__init__()
      self.embedding_layer = tf.keras.layers.Embedding(vocab_length_src, embedding_dimension)
      self.gru = tf.keras.layers.GRU(encoder_units, return_sequences=True, return_state=True, go_backwards = False)  # will have to be replaced with LSTM (LSTM has 3 inputs not 2)

  def call(self, x, last_hidden_state):
    x = self.embedding_layer(x)
    hidden_seguence, hidden_state = self.gru(x, initial_state = last_hidden_state)
    return hidden_seguence, hidden_state

  def initialize_hidden_state(self):
    return tf.zeros((batch_size, encoder_units))

In [ ]:
encoder = Encoder()

In [ ]:
dataset = get_dataset()
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([256, 16]), TensorShape([256, 23]))

In [ ]:
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (256, 16, 128)
Encoder Hidden state shape: (batch size, units) (256, 128)


### Attention Layer


In [ ]:
class Attention(tf.keras.layers.Layer):
  def __init__(self):
    super(Attention, self).__init__()
    self.A = tf.keras.layers.Dense(decoder_units)
    self.B = tf.keras.layers.Dense(encoder_units)
    self.v = tf.keras.layers.Dense(1)

  def call(self, encoder_output, decoder_input):
    # add sequence dimension to decoder input
    decoder_input = tf.expand_dims(decoder_input, 1)
    # sigma(Ax + Bs)
    attention_weights = tf.nn.tanh(self.A(encoder_output) + self.B(decoder_input))
    attention_weights = self.v(attention_weights)
    attention_weights = tf.nn.softmax(attention_weights, axis=1)
    context_vector = attention_weights * encoder_output
    return tf.reduce_sum(context_vector, axis=1)

In [ ]:
attention_layer = Attention()
attention_result = attention_layer(sample_output, sample_hidden)
print("Attention result shape: (batch size, units) {}".format(attention_result.shape))

Attention result shape: (batch size, units) (256, 128)


### Decoder
Just as with the encoder the decoder uses GRU instead of LSTM rn, which has to be changed

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding_layer = tf.keras.layers.Embedding(vocab_length_tar, embedding_dimension)
    self.gru = tf.keras.layers.GRU(decoder_units, return_sequences=True, return_state=True)  # has to be replaced with LSTM
    self.attention_layer = Attention()
    self.output_layer = tf.keras.layers.Dense(vocab_length_tar)

  def call(self, x, last_hidden_state, encoder_output):
    x = self.embedding_layer(x)
    hidden_sequence, hidden_state = self.gru(x, initial_state = last_hidden_state)
    c = self.attention_layer(encoder_output, hidden_state)
    concat = tf.concat([c, hidden_state], axis = 1)
    output = self.output_layer(concat)
    return output, hidden_state

  def initialize_hidden_state(self):
    return tf.zeros((batch_size, decoder_units))

In [ ]:
decoder = Decoder()

In [ ]:
sample_decoder_output, sample_decoder_hidden = decoder(tf.random.uniform((batch_size, 1)), sample_hidden, sample_output)

print ('Decoder pred shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
print ('Decoder hidden shape: (batch_size, vocab size) {}'.format(sample_decoder_hidden.shape))

Decoder pred shape: (batch_size, vocab size) (256, 4859)
Decoder hidden shape: (batch_size, vocab size) (256, 128)


# Training

### Checkpoints

In [ ]:
checkpoint_prefix = os.path.join(path_main_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [ ]:
def restore_checkpoint():
  checkpoint.restore(tf.train.latest_checkpoint(path_main_dir))

### Loss Function

In [ ]:
def loss_function2(lbl, pred):
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(lbl, pred)
    eos = tf.math.not_equal(lbl, 0)     # eos is false for all non tokens
    # potential problem: sentences that are too long are not being penalized
    loss = tf.boolean_mask(loss, eos)
    mean = tf.reduce_mean(loss)
    return mean

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

### Training Step

In [ ]:
@tf.function
def train_step(src, tar, encoder_hidden, decoder_hidden):
  loss = 0
  with tf.GradientTape() as tape:
    encoder_hidden_sequence, encoder_last_hidden_state = encoder(src, encoder_hidden)
    decoder_input = tf.expand_dims([tar_dict.word_index['<s>']] * batch_size, 1)
    
    for i in range(1, max_tar_len):
      pred, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hidden_sequence)
      loss += loss_function(tar[:, i], pred)
      decoder_input = tf.expand_dims(tar[:, i], 1)
      
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
  return loss / max_tar_len
  # max tar len is not a tensor! -> potential slowdown

In [ ]:
#train_step(example_input_batch, example_target_batch, encoder.initialize_hidden_state(), decoder.initialize_hidden_state())

### Train

In [ ]:
total_batches = len(src_train) // batch_size
steps_per_epoch = total_batches

In [ ]:
print(total_batches)

39


In [ ]:
def train_model(starting_epoch=0, final_epoch=10):
  if starting_epoch > 0:
    restore_checkpoint()

  for epoch in range(starting_epoch, final_epoch):
    s = time.time()
    total_loss = 0
    # initialized here instead of train_step for performance reasons
    encoder_initial_hidden_state = encoder.initialize_hidden_state()
    decoder_initial_hidden_state = decoder.initialize_hidden_state()

    for i, (src, tar) in enumerate(dataset.take(steps_per_epoch)):
      batch_loss = train_step(src, tar, encoder_initial_hidden_state, decoder_initial_hidden_state)
      total_loss += batch_loss
      if i % 50 == 0: 
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, i, batch_loss.numpy()))
      
    checkpoint.save(file_prefix = checkpoint_prefix)
    print('\nEpoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n\n'.format(time.time() - s))

*run this cell to train model:*

In [ ]:
print(starting_epoch)
print(final_epoch)

5
5


In [ ]:
train_model(starting_epoch = starting_epoch, final_epoch = final_epoch)

# Inference

### Evaluate Sentence

evaluate gets a list with a single tokenized sentence in it as input

***todo***: has to be changed to matrix input

***todo***: convert greedy search to beam search

***todo***: convert from eager to tf function


In [ ]:
def evaluate(sentence):
  src = tf.convert_to_tensor(sentence)

  encoder_initial_hidden = [tf.zeros((1, encoder_units))]
  encoder_hidden_sequence, encoder_last_hidden = encoder(src, encoder_initial_hidden)

  decoder_hidden = [tf.zeros((1, decoder_units))]
  decoder_input = tf.expand_dims([tar_dict.word_index['<s>']], 0)
  
  pred = tf.zeros(vocab_length_tar)
  
  prob = 0

  res = []

  for i in range(max_tar_len):
    pred, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_hidden_sequence)
    pred = tf.argmax(pred[0]).numpy()
    res += [pred]
    decoder_input = tf.expand_dims([pred], 0)

    # stop unrolling
    if tar_dict.index_word[pred] == '</s>':
      return res

  return res

In [ ]:
def evaluate_beam_search(sentence):
  pass

In [ ]:
example_sentence = "<s> personen stehen vor dem café und sind dabei , einzutreten . </s>"
example_sentence_tokenized = src_dict.texts_to_sequences([example_sentence])
print(example_sentence_tokenized)
evaluate(example_sentence_tokenized)

[[1, 42, 50, 26, 30, 451, 13, 124, 356, 21, 6907, 3, 2]]


[16, 28, 47, 10, 16, 5, 16, 93, 14, 113, 4, 3]

### Evaluate List

evaluate_list gets a list of non-tokenized sentences as input and returns a list of non-tokenized translations as output

In [ ]:
def evaluate_list(text):
  return [evaluate(src_dict.texts_to_sequences([sentence])) for sentence in text]

In [ ]:
example_text = [example_sentence]
print(example_text)
evaluate_list(example_text)

['<s> personen stehen vor dem café und sind dabei , einzutreten . </s>']


[[16, 28, 47, 10, 16, 5, 16, 93, 14, 113, 4, 3]]

### Translate

translate_sentence for testing purposses only

In [ ]:
def translate_sentence(sentence):
  # bpe
  sentence = "<s> " + sentence + " </s>"
  t = evaluate(src_dict.texts_to_sequences([example_sentence]))
  t = tar_dict.sequences_to_texts([t])[0]
  return t

In [ ]:
example_sentence = "personen stehen vor dem café und sind dabei , einzutreten ."
print(src_dict.texts_to_sequences([example_sentence]))
translate_sentence(example_sentence)

[[42, 50, 26, 30, 451, 13, 124, 356, 21, 6907, 3]]


'people standing outside of a cafe about to go in the background . </s>'

translate gets list of non preprocessed, non-tokenized text as input

In [ ]:
def strip_start_token(text):
  t = [" ".join(s.split()[1:]) for s in text]
  return t

In [ ]:
def strip_end_token(text):
  t = [" ".join(s.split()[:-1]) for s in text]
  return t

In [ ]:
def translate(text):
  t = [translate_sentence(sentence) for sentence in text]
  return strip_end_token(t)

In [ ]:
translate([example_sentence])

['people standing outside of a cafe about to go in the background .']

# Evaluation

### BLEU

In [ ]:
def n_gram(reference, hypothesis, n=4):
    # returns the clipped count of n-grams in the hypothesis that match a n-gram in the reference
    # can be used to count the total amount of n-grams in a String S by calling n_grams(S, S)
    reference = reference.split()
    hypothesis = hypothesis.split()
    reference_n_grams = []
    hypothesis_n_grams = []
    ret = 0

    # creating n-gram lists
    for i in range(len(reference) - n + 1):
        tmp = []
        for j in range(n):
            tmp.append(reference[i + j])
        reference_n_grams.append(tmp)

    for i in range(len(hypothesis) - n + 1):
        tmp = []
        for j in range(n):
            tmp.append(hypothesis[i + j])
        hypothesis_n_grams.append(tmp)

    # clipped count
    for n_gram in hypothesis_n_grams:
        if n_gram in reference_n_grams:
            ret += 1
            reference_n_grams.remove(n_gram)  # the remove command in python removes the first instance that is found

    return ret

In [ ]:
def modified_n_gram_precision(L, n=4):  # each element in L has form (reference, hypothesis)
    counter = 0
    denominator = 0
    for pair in L:
        counter += n_gram(pair[0], pair[1], n)      # clipped count of matching n-grams
        denominator += n_gram(pair[1], pair[1], n)  # total amount of n-grams in the hypothesis
    if denominator != 0:
        return counter/denominator
    else:
        return -1
        # this should never be returned as long as there is at least one sentence in the hypothesis that is n long

In [ ]:
def brevity_penalty(c, r):
    """     c := hypothesis length
            r := reference length
    """
    if c > r:
        return 1
    else:
        return math.exp(1-r/c)

### Score

In [ ]:
def bleu_score(hyp, ref, n=4):  # each element in L has form (reference, hypothesis)
    L = []
    for t, d in zip(ref, hyp):
        L.append([t, d])
    c = 0
    r = 0
    s = 0
    for pair in L:
        c += len(pair[1].split())
        r += len(pair[0].split())
    for i in range(1, n + 1):
        tmp = modified_n_gram_precision(L, i)
        if tmp != 0:    # the lecture didn't define what to do in this case, so I just set it to 0
            s += (1/n) * math.log(tmp)
    return brevity_penalty(c, r) * math.exp(s)

### Run

Translate text calculate its BLEU score and save it

In [ ]:
limit_val = 100

In [ ]:
def translate_and_save(src, ref):
  hyp = translate(src)

  ref = strip_start_token(ref)
  ref = strip_end_token(ref)

  bs = bleu_score(hyp, ref)
  print("bleu score = " + bs)
  # todo save


In [ ]:
src = src_val_txt[:limit_val]
src = strip_start_token(ref)
s = strip_end_token(ref)


hyp = translate(src_val_txt[:limit_val])
hyp = strip_end_token(hyp)

ref = tar_val_txt[:limit_val]
ref = strip_start_token(ref)
ref = strip_end_token(ref)


In [ ]:
print(src_val_txt[:10])

In [ ]:
hyp[:10]

In [ ]:
ref[:10]